In [ ]:
library(tidyverse)

In [ ]:
clinical_data_path <- "/Users/shirin/Desktop/Immunarch/downsamplign/Projetcs/INSPIRE/clinical_data/coding_ref"
clinical_data_fname <- "INSPIRE_ClinicalData_CodingRef.xlsx"

clinical_data <- readxl::read_xlsx(
        file.path (
                clinical_data_path , clinical_data_fname))%>%
        dplyr::select(Patient_id ,  
                      `Best response longevity` , 
                      COHORT ,
                      `Histologic diagnosis` ,
                      `Best response cycle`)

In [ ]:
fmm_path <- "/Users/shirin/Desktop/Immunarch/downsamplign/Projetcs/INSPIRE/Stats_Consultation/August_Revision"
fmm_fname <- "INSPIRE_PBMC_longitudinal_within-patient_FMMcluster_2023August09.xlsx"

fmm_data <- readxl::read_xlsx(
        file.path (
                fmm_path , fmm_fname))

In [ ]:
input_path <- "/Users/shirin/Desktop/Immunarch/downsamplign/Projetcs/INSPIRE/dna/pbmc_feb/pbmc_diversity_profiles"
input_fname <- "INSPIRE_PBMC_Diversity_Downsampled.csv"

diversity_indices <- readr::read_csv(file.path (
        input_path , input_fname) ) %>% 
        filter (Locus == "CLONES_TRB") %>%
        filter(Order_q == 1 ) %>%
        dplyr::select(Patient_id , Cycle , Cycle_Code ,  Diversity) %>%
        left_join(clinical_data ,
                  by = "Patient_id")%>%
        mutate( COHORT = fct_relevel( COHORT , 
                                      "SCCHN", 
                                      "TNBC",
                                      "HGSOC",
                                      "MM", 
                                      "MST" )) %>%
        mutate( Cycle = fct_relevel(Cycle ,
                                    "SB" , "C3B" , "C6B" , "C9B" ,
                                    "C12B" , "C15B" , "C18B" , "C21B" ,
                                    "C24B" , "C27B" , "C30B" , "C33B" ,  
                                    "EOTB"))

In [ ]:
color_pal = tibble(response = 
                           c ("CR",
                              "PR",
                              "SD, 6 < n ICB cycles",
                              "SD, n < 6 ICB cycles",
                              "PD" ,
                              "NE"),
                   color = 
                           c ("#033483" ,
                                        "#A7D2E9" ,
                                        "#7DB290" ,
                                        "#FEA500" ,
                                        "#C5231B" ,
                                        "#AFAFAF"
                           ))

In [ ]:
Patient_with_continued_benefit <- (diversity_indices %>%
                                           group_by(Patient_id) %>%
                                           summarise(n = n()) %>%
                                           ungroup () %>%
                                           arrange(desc (n)) %>%
                                           filter(n > 4) )$Patient_id

In [ ]:
ggsave(
        file.path(
                input_path,
                "INSPIRE_PBMC_TRB_LongTermClinicalBenefit_ShannonDiversityDynamicsPatterns.svg" ),
        device = "svg" ,
        height = 1500,
        width = 4000,
        units = "px",
        bg = "transparent",
        dpi = 320)


ggplot(data = diversity_indices %>%
               filter(Patient_id %in% Patient_with_continued_benefit) %>%
               left_join(fmm_data ,
                         by = "Patient_id")) +
        geom_line(
                aes(
                        x = Cycle_Code * 3 ,
                        y = Diversity ,
                        group = Patient_id ,
                        color =  `Best response longevity`
                ),
                linewidth = 0.2
        ) +
        scale_color_manual(
                values = color_pal$color ,
                breaks = color_pal$response
        ) +
        ylab ("PBMC Shannon diversity") +
        xlab ("Time on-ICB (weeks)") +
        theme_minimal() +
        
        theme(
                panel.grid = element_blank(),
                panel.spacing = unit(1 , units = "line"),
                
                axis.line = element_line(colour = "#000000" , linewidth = 0.1),
                axis.ticks = element_line(color = "#000000" , linewidth = 0.25) ,
                axis.ticks.length = unit(0.25 , "line") ,
                
                axis.title = element_text ( vjust = 2, family = "Helvetica", face="plain" ,
                                            size = 15 , hjust = 0.5 ),
                
                
                axis.text = element_text ( size = 15, family = "Helvetica", face="plain" ,
                                           hjust = 0.5, colour = "#000000"),
                
                
                strip.text = element_text ( size = 15, family = "Helvetica", face="plain" ,
                                           hjust = 0, colour = "#000000"),
                
                legend.position = "none" ,
                
                
                aspect.ratio = 1/1.6 
        )+
        facet_grid(. ~ `cluster based on within-patient variance of Shannon diversity` ,
                   labeller = as_labeller(c(`Cluster I` = "Stable" ,
                                            `Cluster II` = "Sporadic fluctuations to\nstabilization transition" ,
                                            `Cluster III` = "Cyclic fluctuations")))

dev.off()